In [41]:
import os
import pandas as pd
import shutil
import cv2
from tqdm import tqdm
from OCR_XML_to_DataFrame import OCR_XML_to_DataFrame

In [42]:
key_mappings = pd.read_excel("D:/Xelp_work/FSL Project/FSL_Phase_II/Milestone_3/FSL_Forms_Keys.xlsx")
list_columns = key_mappings["Key_Name"].tolist()

In [43]:
def get_missing_tooth_gt(row):
    ground_truth = row["TRANS_Value"]
    return ground_truth

In [44]:
def get_qual_code(row):
    if ("?" not in row["OCR_Optimizer"]) & (row["OCR_Optimizer"] != '[BLANK]'):
        ground_truth = row["OCR_Optimizer"]
    else:
        ground_truth = row["Data"].replace('0', '[BLANK]')
    return ground_truth

In [45]:
def get_org_name(row):
    if ("?" not in row["OCR_Optimizer"]) & (row["OCR_Optimizer"] != '[BLANK]'):
        ground_truth = row["OCR_Optimizer"]
    else:
        ground_truth = row["TRANS_Value"]
    return ground_truth

In [46]:
def get_post_code(row):
    if ("?" not in row["OCR_Optimizer"]) & (row["OCR_Optimizer"] != '[BLANK]'):
        ground_truth = row["OCR_Optimizer"]
    else:
        if row["TRANS_Value"][-4:] == '0000':
            ground_truth = row["TRANS_Value"][:-4]
        else:
            ground_truth = row["TRANS_Value"]
    return ground_truth

In [47]:
def get_charges(row):
    ground_truth = row["TRANS_Value"]
    return ground_truth

In [48]:
def get_relationship_code(row):
    if ("?" not in row["OCR_Optimizer"]) & (row["OCR_Optimizer"] != '[BLANK]'):
        ground_truth = row["OCR_Optimizer"]
    else:
        if str(row["Data"]).strip() in ['01', '18', '19', '21', 'X']:
            ground_truth = 'X'
        else:
            ground_truth = '[BLANK]'
    return ground_truth

In [49]:
def get_Inssex_code(row):
    if row["OCR_Optimizer"] == 'X':
        ground_truth = row["OCR_Optimizer"]
    elif row["OCR_Optimizer"] == '[BLANK]':
        ground_truth = row["TRANS_Value"]
    else:
        ground_truth = row["TRANS_Value"]
    return ground_truth

In [50]:
def get_TransTypeStmt_code(row):
    if row["OCR_Optimizer"] == 'X':
        ground_truth = row["OCR_Optimizer"]
    elif row["OCR_Optimizer"] == '[BLANK]':
        if row["Data"] ==  'Y':
            ground_truth =  'X'
        else:
            ground_truth = '[BLANK]'
    elif "?" in row["OCR_Optimizer"]:
        if row["Data"] ==  'Y':
            ground_truth =  'X'
    else:
        ground_truth = '[BLANK]'
        
    return ground_truth

In [51]:
def get_TransTypeReqforPreAuth_code(row):
    if row["OCR_Optimizer"] == 'X':
        ground_truth = row["OCR_Optimizer"]
    elif row["OCR_Optimizer"] == '[BLANK]':
        if row["Data"] ==  'Y':
            ground_truth =  'X'
        else:
            ground_truth = '[BLANK]'
    elif "?" in row["OCR_Optimizer"]:
        if row["Data"] ==  'Y':
            ground_truth =  'X'
    else:
        ground_truth = '[BLANK]'
        
    return ground_truth

In [52]:
def get_PatAcctNo(row):
    if row["Data"] == '0':
        ground_truth = '[BLANK]'
    else:
        ground_truth = row["TRANS_Value"]
    return ground_truth

In [53]:
def get_DiagCodeQual(row):
    if row["OCR_Optimizer"] in ["[BLANK]", "A", "B", "AB"]:
        ground_truth = row["OCR_Optimizer"]
    else:
        ground_truth = '[BLANK]'
    return ground_truth

In [54]:
def get_Phone_Num(row):
    if ("?" not in row["OCR_Optimizer"]) & (row["OCR_Optimizer"] != '[BLANK]'):
        ground_truth = row["OCR_Optimizer"]
    else:
        row["OCR_Optimizer"] = row["OCR_Optimizer"].replace("?", "")
        ground_truth = row["OCR_Optimizer"]
    return ground_truth    

In [55]:
def get_Prosthesis(row):
    ground_truth = row["Data"]
    return ground_truth

In [56]:
def get_ProcDesc(row):
    if "?" not in row["OCR_Optimizer"]:
        if row["OCR_Optimizer"] != "[BLANK]":
            ground_truth = row["OCR_Optimizer"]
        else:
            ground_truth = row["TRANS_Value"]
    else:
        ground_truth = row["TRANS_Value"]
    return ground_truth

In [57]:
def get_ToothSurface(row):
    if ("?" not in row["OCR_Optimizer"]) & (row["OCR_Optimizer"] != '[BLANK]'):
        ground_truth = row["OCR_Optimizer"]
    else:
        row["TRANS_Value"] = row["TRANS_Value"].replace(":", "")
        ground_truth = row["TRANS_Value"]
    return ground_truth                                           

In [58]:
def get_PayerInfo(row):
    if "?" not in row["OCR_Optimizer"]:
        if row["OCR_Optimizer"] != "[BLANK]":
            ground_truth = row["OCR_Optimizer"]
        else:
            ground_truth = row["TRANS_Value"]
    else:
        row["OCR_Optimizer"] = row["OCR_Optimizer"].replace("?", "")
        ground_truth = row["OCR_Optimizer"]
    return ground_truth

def get_FacProvData(row):
    ground_truth = row["TRANS_Value"]
    return ground_truth

def get_Orthodontics(row):
    ground_truth = row["Data"]
    return ground_truth

def get_AccountNumber(row):
    ground_truth = row["TRANS_Value"]
    return ground_truth

In [59]:
def generate_ground_truth(row):
    ground_truth = ''
    try:
        if row["Field_Name"] == 'DEN_MissingTooth':
            ground_truth = get_missing_tooth_gt(row)
        elif row["Field_Name"] == 'DEN_DiagCodeQual':
            ground_truth = get_qual_code(row)
        elif row["Field_Name"] in ["DEN_FacProvOrgName"]:
            ground_truth = get_org_name(row)
        elif row["Field_Name"] in ["DEN_BillProvPostCode","DEN_FacProvPostCode"]:
            ground_truth = get_post_code(row)
        elif row["Field_Name"] in ["DEN_LineCharges", "DEN_TotalCharges"]:
            ground_truth = get_charges(row)
        elif row["Field_Name"] in ["DEN_PatRelShipSelf","DEN_PatRelShipSpouse","DEN_PatRelShipDep","DEN_PatRelShipOther"]:
            ground_truth = get_relationship_code(row)
        elif row["Field_Name"] in ["DEN_PriInsSex", "DEN_PatSex", "DEN_SecInsSexF", "DEN_SecInsSexM"]:
            ground_truth = get_Inssex_code(row)
        elif row["Field_Name"] == 'DEN_TransTypeStmtOfServ':
            ground_truth = get_TransTypeStmt_code(row)
        elif row["Field_Name"] == 'DEN_TransTypeReqforPreAuth':
            ground_truth = get_TransTypeReqforPreAuth_code(row)
        elif row["Field_Name"] == "DEN_PatAcctNo":
            ground_truth = get_PatAcctNo(row)
        elif row["Field_Name"] == 'DEN_DiagCodeQual':
            ground_truth =  get_DiagCodeQual(row)
        elif row["Field_Name"] in ['DEN_BillProvPhone', 'DEN_FacProvPhone']:
            ground_truth =  get_Phone_Num(row)
        elif row["Field_Name"] in ['DEN_RplProsthesisN','DEN_RplProsthesisY']:
            ground_truth = get_Prosthesis(row)
        elif row["Field_Name"] == 'DEN_ProcDesc':
            ground_truth = get_ProcDesc(row)
        elif row["Field_Name"] == 'DEN_ToothSurface':
            ground_truth = get_ToothSurface(row)
        elif row["Field_Name"] in ['DEN_PayerPostCode', 'DEN_PayerState', 'DEN_PatSignDate', \
                                   'DEN_Encloseure', 'DEN_FacProvSignDate']:
            ground_truth = get_PayerInfo(row)
        elif row["Field_Name"] == "DEN_IsThereSecIns":
            ground_truth = row["TRANS_Value"]
        elif row["Field_Name"] in ["DEN_OrthodonticsN", "DEN_OrthodonticsY"]:
            ground_truth = get_Orthodontics(row)
        elif row["Field_Name"] in ["DEN_FacProvState", "DEN_FacProvCity"]:
            ground_truth = get_FacProvData(row)
        elif row["Field_Name"] == 'DEN_AccountNumber':
            ground_truth = get_AccountNumber(row)
        
        else:
            if ("?" not in row["OCR_Optimizer"]) & (row["OCR_Optimizer"] != '[BLANK]'):
                ground_truth = row["OCR_Optimizer"] # need to check, sometimes, though the values not printed 
            else:
                ground_truth = row["TRANS_Value"]
    except:
        ground_truth = row["TRANS_Value"]
    return ground_truth

In [60]:
def create_annotation_dict(df):
    result_dict = {}
    for index, row in df.iterrows():
        field_name = row['Latest_Keys']
        trans_value = row['Ground_truth']
        if field_name in result_dict:
            result_dict[field_name] = f"{result_dict[field_name]};{trans_value}"
        else:
            result_dict[field_name] = trans_value 
    return result_dict

In [61]:
json_target_path = r'D:\Xelp_work\Ins_Datasets\Ada_data\ADA_Training_data\key'
images_source_path = r'D:\Xelp_work\Ins_Datasets\Ada_data\JPG'
images_target_path = r'D:\Xelp_work\Ins_Datasets\Ada_data\ADA_Training_data\img'

In [62]:
mapping_dict = key_mappings.set_index('Key_Name').to_dict()['Modified_key']

In [63]:
TRANS_Dir = r'D:\Xelp_work\Ins_Datasets\Ada_data\TRANS'

Trans_files = os.listdir(TRANS_Dir)
len(Trans_files)

18848

In [64]:
req_img = os.listdir(images_source_path)
len(req_img)

18298

In [65]:
import json

In [66]:
def post_process_annotation_dict(Annotation_dict):
    if Annotation_dict["18_Self"] == "X":
        Annotation_dict["20_Name"] = Annotation_dict["12_Name"]
        Annotation_dict["20_Address"] = Annotation_dict["12_Address"]
        Annotation_dict["20_City"] = Annotation_dict["12_City"]
        Annotation_dict["20_State"] = Annotation_dict["12_State"]
        Annotation_dict["20_Zip"] = Annotation_dict["12_Zip"]

    if Annotation_dict["22_Sex"] == "F":
        Annotation_dict["22_F"] = "X"
    elif Annotation_dict["22_Sex"] == "M":
        Annotation_dict["22_M"] = "X"
    
    if Annotation_dict["14_Sex"] == "F":
        Annotation_dict["14_F"] = "X"
    elif Annotation_dict["14_Sex"] == "M":
        Annotation_dict["14_M"] = "X"
    
    if '9999' in Annotation_dict["8_SSN_ID"]:
        Annotation_dict["8_SSN_ID"] = '[BLANK]'

    return Annotation_dict
    

In [67]:
import gc
gc.collect()

12954

In [68]:
overall_data = pd.DataFrame()
Multi_ADA = []
c = 0
for each_trans_file in tqdm(Trans_files):
    trans_file_path = os.path.join(TRANS_Dir, each_trans_file)
#     print(trans_file_path)
    grnd_df, single_fields, grp_field_names, table_field_names, image_name = OCR_XML_to_DataFrame(trans_file_path, list_columns)
    if (len(image_name)) > 1:
        Multi_ADA.append(each_trans_file)
#     image_name = image_name[:-4]+"jpg"
#     break
#     try:
#         if image_name in req_img:
# #             print(image_name)
#             grnd_df.fillna("[BLANK]", inplace=True)
#             grnd_df["Ground_truth"] = grnd_df.apply(generate_ground_truth, axis=1)
#             grnd_df['Latest_Keys'] = grnd_df['Field_Name'].map(mapping_dict)
#             Annotation_dict = create_annotation_dict(grnd_df)
            
#             Annotation_dict = post_process_annotation_dict(Annotation_dict)
#             input_image_file = os.path.join(images_source_path, image_name)       
#             try:
#                 img = cv2.imread(input_image_file)
# #                 print(img.shape)
#                 json_file = each_trans_file[:-7]+"0.json"
#                 output_image_path = os.path.join(images_target_path, json_file[:-4]+"jpg")
#                 cv2.imwrite(output_image_path, img)
#                 output_json_path = os.path.join(json_target_path, json_file)
                
#                 grnd_df["Image_Name"] = image_name
#                 grnd_df["XML_File"] = each_trans_file
#     #             overal_data = pd.concat([overal_data, grnd_df], ignore_index=True)
#             except Exception as p:
#                 raise p
                
#             with open(output_json_path, "w") as output_file:
#                 json.dump(Annotation_dict, output_file)
#     except Exception as e:
#         raise e
        
#     c += 1
# #     print(Annotation_dict)
#     if c %200 == 0:
#         gc.collect()

  0%|          | 48/18848 [00:07<47:11,  6.64it/s]


KeyboardInterrupt: 

In [ ]:
len(Multi_ADA)

458

In [ ]:
# Assuming Multi_ADA is a list of strings or a string
with open("Multi_ada.txt", "w") as file:
    file.writelines("\n".join(Multi_ADA))


In [ ]:
import os
import json

# Define the path to the folder containing the JSON files
folder_path = r'D:\Xelp_work\Ins_Datasets\Ada_data\ADA_Training_data\key'

# Iterate over each file in the folder
for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)
        
        # Read the JSON file
        with open(file_path, "r") as file:
            data = json.load(file)

        # Modify the data (extract only the first 10 values before ";" occurs)
        for key, value in data.items():
            values = value.split(";")
            extracted_values = [v.strip() for v in values if v.strip()][:10]
            extracted_values_str = ";".join(extracted_values)
            data[key] = extracted_values_str

        # Save the modified data back to the JSON file
        with open(file_path, "w") as file:
            json.dump(data, file, indent=4)

print("All files processed.")


100%|███████████████████████████████████████████████████████████████████████████| 18298/18298 [00:22<00:00, 828.00it/s]

All files processed.


In [ ]:
import os
import json
import pandas as pd

# Define the path to the folder containing the modified JSON files
folder_path = r'D:\Xelp_work\Ins_Datasets\Ada_data\ADA_Training_data\key'

# List to store dictionaries from each JSON file
json_data_list = []

# Iterate over each file in the folder
for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)
        
        # Read the JSON file into a dictionary
        with open(file_path, "r") as file:
            data = json.load(file)
            # Add the file name as a column in the dictionary
            data["file_name"] = filename
            json_data_list.append(data)

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(json_data_list)


100%|██████████████████████████████████████████████████████████████████████████| 18298/18298 [00:02<00:00, 6120.17it/s]


In [ ]:
for each_column in tqdm(df.columns):
    print(each_column, len(df[df[each_column] == '[BLANK]']), len(df[df[each_column] != '[BLANK]']) )

  2%|█▍                                                                                | 2/119 [00:00<00:12,  9.24it/s]

15_SSN_ID 0 18298
2_Pre_Auth_Num 17782 516



  3%|██▊                                                                               | 4/119 [00:00<00:11, 10.24it/s]

3_Company 0 18298
3_Lname 18298 0
3_FName 18298 0


  6%|████▊                                                                             | 7/119 [00:00<00:11,  9.64it/s]

...

45_Auto_accident 18298 0
45_Other_accident 18298 0
14_Sex 0 18298


 97%|█████████████████████████████████████████████████████████████████████████████▉  | 116/119 [00:12<00:00, 10.55it/s]

14_M 8110 10188
14_F 9402 8896
34_ICDInd 18298 0


100%|████████████████████████████████████████████████████████████████████████████████| 119/119 [00:12<00:00,  9.62it/s]

16_Grp_Num 3634 14664
35_NEA 18152 146
file_name 0 18298


In [ ]:
cols_to_check = ['2_Pre_Auth_Num', '5_Name','8_SSN_ID', '9_Plan_Number','17_Name', '42_Months_remain', '41_Date', '44_Date', '46_Date', \
                 '47_Auto', '1_TypeEPSDT', '31A_Other', '48_Prefix', '43_Yes', '19_Code_FT', '19_Code_PT', \
                 '45_illness/injury','33_Missing_Teeth', '35_NEA', '52_Fac_Other_ID', '34_D_Code', '7_M', '10_Other', '58_Fac_ProvIDFull']

train_images_list = []
for index, row in df.iterrows():
    # Check if any value in the specified columns is not equal to '[BLANK]'
    if any(row[col] != '[BLANK]' for col in cols_to_check):
        # If condition is true, add the entire row to train_images_list
        train_images_list.append(row["file_name"])

# Print the number of rows added to train_images_list
print(f"Number of rows added to train_images_list: {len(train_images_list)}")


Number of rows added to train_images_list: 14520


In [ ]:
len(train_images_list)

14520

In [ ]:
train_images_list[0]

'20220231172307627_000.json'

In [ ]:
json_source_dir = r'D:\Xelp_work\Ins_Datasets\Ada_data\ADA_Training_data\key'
img_source_dir = r'D:\Xelp_work\Ins_Datasets\Ada_data\ADA_Training_data\img'

In [ ]:
tr_img_target_dir =  r'D:\Xelp_work\Ins_Datasets\Ada_data\ADA_Training_data\train\img'
tr_json_target_dir = r'D:\Xelp_work\Ins_Datasets\Ada_data\ADA_Training_data\train\key'

vl_img_target_dir =  r'D:\Xelp_work\Ins_Datasets\Ada_data\ADA_Training_data\val\img'
vl_json_target_dir = r'D:\Xelp_work\Ins_Datasets\Ada_data\ADA_Training_data\val\key'

test_img_target_dir =  r'D:\Xelp_work\Ins_Datasets\Ada_data\ADA_Training_data\test\img'
test_json_target_dir = r'D:\Xelp_work\Ins_Datasets\Ada_data\ADA_Training_data\test\key'

os.makedirs(tr_img_target_dir, exist_ok= True)
os.makedirs(tr_json_target_dir, exist_ok= True)
os.makedirs(vl_img_target_dir, exist_ok= True)
os.makedirs(vl_json_target_dir, exist_ok= True)
os.makedirs(test_img_target_dir, exist_ok= True)
os.makedirs(test_json_target_dir, exist_ok= True)

In [ ]:
import shutil


In [ ]:
for each_file_name in tqdm(train_images_list):
    source_im_path = os.path.join(img_source_dir, each_file_name[:-4]+"jpg")
    source_json_path = os.path.join(json_source_dir, each_file_name)
    try:
        shutil.move(source_im_path, tr_img_target_dir)
        shutil.move(source_json_path, tr_json_target_dir)
    except:
        pass
#     break

100%|███████████████████████████████████████████████████████████████████████████| 14520/14520 [00:14<00:00, 968.55it/s]


In [ ]:
df = df.sort_values(by='15_SSN_ID')

In [ ]:
df["15_SSN_ID"].nunique()

17094

In [ ]:
import os

# jpg_folder = r'D:\Xelp_work\Ins_Datasets\Ada_data\ADA_Training_data\train\img'
# json_folder = r'D:\Xelp_work\Ins_Datasets\Ada_data\ADA_Training_data\train\key'

jpg_files = [os.path.splitext(f)[0] for f in os.listdir(test_img_target_dir) if f.endswith('.jpg')]
json_files = [os.path.splitext(f)[0] for f in os.listdir(test_json_target_dir) if f.endswith('.json')]


print(len(jpg_files))
print(len(json_files))

print(jpg_files[0], json_files[0])

2578
2578
20220231172307626_000 20220231172307626_000


In [ ]:
missing_in_lst  = (set(json_files)).difference(set(jpg_files)) 
print(missing_in_lst)


missing_in_Excel  = (set(jpg_files)).difference(set(json_files))
print(missing_in_Excel)

set()
set()


In [ ]:
df

,15_SSN_ID,2_Pre_Auth_Num,3_Company,3_Lname,3_FName,3_Address,3_City,3_State,3_Zip,3_Code_ext,...,45_illness/injury,45_Auto_accident,45_Other_accident,14_Sex,14_M,14_F,34_ICDInd,16_Grp_Num,35_NEA,file_name
12288,0,[BLANK],CIGNA,[BLANK],[BLANK],PO BOX 188037,CHATTANOOGA,TN,37422,[BLANK],...,[BLANK],[BLANK],[BLANK],M,X,[BLANK],[BLANK],0631202,[BLANK],20220231352315604_000.json
9033,000000000,[BLANK],CIGNA DENTAL,[BLANK],[BLANK],PO BOX 188037,[BLANK],TN,374228037,[BLANK],...,[BLANK],[BLANK],[BLANK],M,X,[BLANK],[BLANK],[BLANK],[BLANK],20220231282306420_000.json
13728,000000000,[BLANK],CIGNA,[BLANK],[BLANK],[BLANK],CHATTANOOGA,TN,374227223,[BLANK],...,[BLANK],[BLANK],[BLANK],M,X,[BLANK],[BLANK],3210196,[BLANK],20220231362305374_000.json
14716,000001153,[BLANK],CIGNA,[BLANK],[BLANK],PO BOX 188037,[BLANK],TN,374228037,[BLANK],...,[BLANK],[BLANK],[BLANK],M,X,[BLANK],[BLANK],3338705,[BLANK],20220231372304917_000.json
9543,000002040,[BLANK],CIGNA HEALTHCARE SERVICE CENTER,[BLANK],[BLANK],[BLANK],CHATTANOOGA,TN,374228037,[BLANK],...,[BLANK],[BLANK],[BLANK],M,X,[BLANK],[BLANK],[BLANK],[BLANK],20220231322309186_000.json
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13495,XXA982633031,[BLANK],BLUE CROSS,[BLANK],[BLANK],PO BOX 986005,BOSTON,MA,02298,[BLANK],...,[BLANK],[BLANK],[BLANK],F,[BLANK],X,[BLANK],[BLANK],[BLANK],20220231362304085_000.json
15794,YFW073W02291,[BLANK],ANTHEM DENTAL,[BLANK],[BLANK],PO BOX 1115,MINNEAPOLIS,MN,554401115,[BLANK],...,[BLANK],[BLANK],[BLANK],M,X,[BLANK],[BLANK],[BLANK],[BLANK],20220231372309355_000.json
11548,YPU633735100,[BLANK],MAESTRO HEALTH,[BLANK],[BLANK],PO BOX 188061,CHATTANOOGA,TN,37422,[BLANK],...,[BLANK],[BLANK],[BLANK],F,[BLANK],X,[BLANK],[BLANK],[BLANK],20220231352310011_000.json
2615,Z8421862103,[BLANK],CIGNA DHMO,[BLANK],[BLANK],PO BOX 188045,CHATTANOOGA,TN,37422,[BLANK],...,[BLANK],[BLANK],[BLANK],F,[BLANK],X,[BLANK],[BLANK],[BLANK],20220231212322994_000.json


In [ ]:
remaining = df[~df["file_name"].isin(os.listdir(tr_json_target_dir))]

In [ ]:
remaining["15_SSN_ID"].nunique()

3541

In [ ]:
remaining = remaining.sort_values(by='15_SSN_ID')
remaining

,15_SSN_ID,2_Pre_Auth_Num,3_Company,3_Lname,3_FName,3_Address,3_City,3_State,3_Zip,3_Code_ext,...,45_illness/injury,45_Auto_accident,45_Other_accident,14_Sex,14_M,14_F,34_ICDInd,16_Grp_Num,35_NEA,file_name
10833,000002720,[BLANK],CIGNA DENTAL,[BLANK],[BLANK],PO BOX 188037,CHATTANOOGA,TN,374228037,[BLANK],...,[BLANK],[BLANK],[BLANK],M,X,R,[BLANK],3338705,[BLANK],20220231332314938_000.json
6851,000002935,[BLANK],CIGNA,[BLANK],[BLANK],[BLANK],[BLANK],TN,[BLANK],[BLANK],...,[BLANK],[BLANK],[BLANK],M,X,[BLANK],[BLANK],3342018,[BLANK],20220231252309370_000.json
7583,000003227,[BLANK],CIGNA,[BLANK],[BLANK],PO BOX 188037,CHATTANOOGA,TN,37422,[BLANK],...,[BLANK],[BLANK],[BLANK],F,[BLANK],X,[BLANK],[BLANK],[BLANK],20220231262308191_000.json
14201,000007436,[BLANK],CIGNA,[BLANK],[BLANK],PO BOX 188037,CHATTANOOGA,TN,[BLANK],[BLANK],...,[BLANK],[BLANK],[BLANK],M,X,[BLANK],[BLANK],[BLANK],[BLANK],20220231362307016_000.json
10425,000019199,[BLANK],CIGNA,[BLANK],[BLANK],[BLANK],[BLANK],[BLANK],[BLANK],[BLANK],...,[BLANK],[BLANK],[BLANK],F,[BLANK],X,[BLANK],3341916,[BLANK],20220231332312955_000.json
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9414,U90150512,[BLANK],CIGNA,[BLANK],[BLANK],[BLANK],[BLANK],[BLANK],[BLANK],[BLANK],...,[BLANK],[BLANK],[BLANK],F,[BLANK],X,[BLANK],[BLANK],[BLANK],20220231282308519_000.json
13727,W262830621,[BLANK],CIGNA HEALTHCARE OF TN,[BLANK],[BLANK],PO BOX 182223,CHATTANOOGA,TN,37422,[BLANK],...,[BLANK],[BLANK],[BLANK],M,X,[BLANK],[BLANK],[BLANK],[BLANK],20220231362305373_000.json
3949,W270116903,[BLANK],AETNA,[BLANK],[BLANK],[BLANK],[BLANK],KY,405124079,[BLANK],...,[BLANK],[BLANK],[BLANK],F,[BLANK],X,[BLANK],[BLANK],[BLANK],20220231222309473_000.json
6766,W279391027,[BLANK],AETNA,[BLANK],[BLANK],[BLANK],[BLANK],KY,[BLANK],[BLANK],...,[BLANK],[BLANK],[BLANK],M,X,[BLANK],[BLANK],[BLANK],[BLANK],20220231252308968_000.json


In [ ]:
validation_split = remaining[:1200]
test_split =  remaining[1200:]

In [ ]:
validation_split["15_SSN_ID"].nunique()
test_split["15_SSN_ID"].nunique()

2408

In [ ]:
vl_img_target_dir =  r'D:\Xelp_work\Ins_Datasets\Ada_data\ADA_Training_data\val\img'
vl_json_target_dir = r'D:\Xelp_work\Ins_Datasets\Ada_data\ADA_Training_data\val\key'

test_img_target_dir =  r'D:\Xelp_work\Ins_Datasets\Ada_data\ADA_Training_data\test\img'
test_json_target_dir = r'D:\Xelp_work\Ins_Datasets\Ada_data\ADA_Training_data\test\key'

In [ ]:
val_images_list = validation_split["file_name"].tolist()
test_images_list = test_split["file_name"].tolist()

In [ ]:
for each_file_name in tqdm(test_images_list):
    source_im_path = os.path.join(img_source_dir, each_file_name[:-4]+"jpg")
    source_json_path = os.path.join(json_source_dir, each_file_name)
    try:
        shutil.move(source_im_path, test_img_target_dir)
        shutil.move(source_json_path, test_json_target_dir)
    except:
        pass
#     break

100%|█████████████████████████████████████████████████████████████████████████████| 2579/2579 [00:02<00:00, 959.82it/s]


In [91]:
image_dir = r'D:\Xelp_work\Ins_Datasets\Ada_data\ADA_Training_data\test\img'
key_dir = r'D:\Xelp_work\Ins_Datasets\Ada_data\ADA_Training_data\test\key'
with open("Multi_ada.txt", "r") as file:
    file_list = file.readlines()
    for line in file_list:
        line = line.strip()  # Remove leading/trailing whitespaces and newlines
        image_name = line.split(".")[0] + ".jpg"
        json_name = line.split(".")[0] + ".json"
        image_path = os.path.join(image_dir, image_name)
        json_path = os.path.join(key_dir, json_name)
        print(image_path, json_path)
        try:
            os.remove(image_path)
            os.remove(json_path)
            print("Deleted")
        except:
            pass
        

D:\Xelp_work\Ins_Datasets\Ada_data\ADA_Training_data\test\img\20220231192308248_000.jpg D:\Xelp_work\Ins_Datasets\Ada_data\ADA_Training_data\test\key\20220231192308248_000.json
D:\Xelp_work\Ins_Datasets\Ada_data\ADA_Training_data\test\img\20220231212306063_000.jpg D:\Xelp_work\Ins_Datasets\Ada_data\ADA_Training_data\test\key\20220231212306063_000.json
D:\Xelp_work\Ins_Datasets\Ada_data\ADA_Training_data\test\img\20220231212306065_000.jpg D:\Xelp_work\Ins_Datasets\Ada_data\ADA_Training_data\test\key\20220231212306065_000.json
D:\Xelp_work\Ins_Datasets\Ada_data\ADA_Training_data\test\img\20220231212307146_000.jpg D:\Xelp_work\Ins_Datasets\Ada_data\ADA_Training_data\test\key\20220231212307146_000.json
D:\Xelp_work\Ins_Datasets\Ada_data\ADA_Training_data\test\img\20220231212307148_000.jpg D:\Xelp_work\Ins_Datasets\Ada_data\ADA_Training_data\test\key\20220231212307148_000.json
D:\Xelp_work\Ins_Datasets\Ada_data\ADA_Training_data\test\img\20220231212307220_000.jpg D:\Xelp_work\Ins_Datasets\A

In [80]:
file_list

'20220231192308248_000.TRANS\n'

In [71]:
for each_multi in Multi_ADA:
    image_name = each_multi.split(".")[0] + ".jpg"
    json_name = each_multi.split(".")[0] + ".json"
    print(image_name, json_name)

20220231192308248_000.jpg 20220231192308248_000.json
